<a href="https://colab.research.google.com/github/phucb2/lm-hackers/blob/main/Gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-01 16:13:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.007s  

2024-01-01 16:13:38 (162 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
!ls input.txt

input.txt


In [ ]:
!head input.txt

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:


In [ ]:
with open('input.txt') as file:
  content = file.read()

In [ ]:
len(content)

1115394

In [ ]:
chars = sorted(set(list(content)))

In [ ]:
len(chars)

65

In [ ]:
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [ ]:
stoi = {c:i for i, c in enumerate(chars)}
itoc= {i:c for i, c in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s ]
decode = lambda e: ''.join([itoc[i] for i in e])

In [ ]:
encode("hello")

[46, 43, 50, 50, 53]

In [ ]:
decode([46, 43, 50, 50, 53])

'hello'

In [ ]:
import torch

tensor = torch.tensor(encode(content), dtype=torch.long)
tensor[:10]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [ ]:
train_sz = int(len(tensor)*0.9)

In [ ]:
train_ts = tensor[:train_sz]
valid_ts = tensor[train_sz:]

In [ ]:
block_size = 8
train_ts[:block_size+1].shape

torch.Size([9])

In [ ]:
train_ts[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_ts[:block_size]
y = train_ts[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when context {context} target is {target}")

when context tensor([18]) target is 47
when context tensor([18, 47]) target is 56
when context tensor([18, 47, 56]) target is 57
when context tensor([18, 47, 56, 57]) target is 58
when context tensor([18, 47, 56, 57, 58]) target is 1
when context tensor([18, 47, 56, 57, 58,  1]) target is 15
when context tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when context tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [ ]:
torch.randint(train_ts.shape[0] - block_size, (4,))

tensor([965347, 226082,  61742, 898828])

In [ ]:
torch.manual_seed(203)
batch_size = 4
block_size = 8

def get_batch(ds:str = 'train'):
  data = train_ts if ds == 'train' else valid_ts
  ix = torch.randint(data.shape[0] - block_size, (batch_size,))
  xb = torch.stack([data[i:i+block_size] for i in ix])
  yb = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return xb, yb

In [ ]:
get_batch()

(tensor([[ 0, 32, 46, 53, 59,  1, 61, 47],
         [ 1, 58, 46, 43,  1, 51, 39, 58],
         [58, 46,  1, 47, 52,  1, 44, 53],
         [43, 57, 57, 43, 42,  1, 63, 53]]),
 tensor([[32, 46, 53, 59,  1, 61, 47, 50],
         [58, 46, 43,  1, 51, 39, 58, 58],
         [46,  1, 47, 52,  1, 44, 53, 56],
         [57, 57, 43, 42,  1, 63, 53, 59]]))

In [ ]:
vocab_size = len(chars)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLM(nn.Module):
  def __init__(self):
    super().__init__()
    self.embd = nn.Embedding(vocab_size, vocab_size)
  def forward(self, idx, targets=None):
    logits = self.embd(idx) # (B, T, C) and C = vocab_size
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # (B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss
  def generate(self,idx,max_generate):
    for _ in range(max_generate):
      logits, _ = self(idx) # B, T, C
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim=-1) # B, C
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1) # B, T+1
    return idx


bigramLM = BigramLM()
sample = get_batch()
o,l  = bigramLM(*sample)
(o.shape, l.shape)

(torch.Size([32, 65]), torch.Size([]))

In [ ]:
l

tensor(4.6769, grad_fn=<NllLossBackward0>)

In [ ]:
s = torch.zeros((1, 1), dtype=torch.long)
g = bigramLM.generate(s, 100)

In [ ]:
decode(list(g[0].tolist()))

'\nbJfZSM:vobzxEbnA&Hc!\ne&aXxgZDRfYQHzI;-Sc!IPPVICsbQQe&PVHzdb&ct$bQsMp;.qVgHHwe$xQe  ZloaEcTbxETp,d&.T'

In [ ]:
def generate_text(m, max_size):
  init = torch.zeros((1, 1), dtype=torch.long)
  o = m.generate(init, max_size)
  return decode(list(o[0].tolist()))

In [ ]:
batch_size = 32
epoches = 10000

m = BigramLM()
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
for _ in range(epoches):
  xb, yb = get_batch('train')
  optimizer.zero_grad(set_to_none=True)
  logits, loss = m(xb, yb)
  loss.backward()
  optimizer.step()
print(loss.item())

2.497217893600464


In [ ]:
print(generate_text(m, 500))


She au g angero us y ais ft ive
Uy aitllare bly?
CUSird, ond&, bre y w, thes wory blereisuneesho t d, pest, ur!
QUSathe m'diquthey ges,

-at rd prik s.
ghcyodio we, se st he jurslelikXmme army:
Ar t idoupas t? ENG than,
HUnowicice horoferexalan! be at Marbll?
LORICI k. f ais ve ujanoure'thelithail th s Sofr nie inno w; s eckeryo t p s all3Jcanof wErre te ondanee lin l'che Qhe bers st woroupake tousswavengr lonse.
Thoor in
Highofasas, or vefawbe fas gere, ne f id kie He ir.

Cos an OLABu RKIOMES:


In [ ]:
torch.manual_seed(2024)

In [ ]:
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
x[0]

tensor([[-1.2262, -0.0093],
        [ 1.5420, -0.4657],
        [ 0.2795, -0.2610],
        [ 0.6230, -1.1561],
        [ 0.1171, -1.8865],
        [ 2.1822, -0.1930],
        [ 0.5358, -0.8898],
        [-0.3099,  0.7741]])

In [ ]:
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] # (t, C)
    xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, 1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [ ]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei = wei / torch.sum(wei, 1, keepdim=True)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [ ]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
torch.allclose(xbow, xbow2)

True

In [ ]:
xbow.shape

torch.Size([4, 8, 2])

In [ ]:
a = torch.tril(torch.ones(3, 3))
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
a / torch.sum(a, 1, keepdim=True)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [ ]:
b = torch.randint(0, 10, (3, 2)).float()

In [ ]:
b

tensor([[8., 4.],
        [7., 9.],
        [3., 8.]])

In [ ]:
a @ b

tensor([[ 8.,  4.],
        [15., 13.],
        [18., 21.]])

In [ ]:
x

tensor([[[ 0.3171, -1.8716],
         [-0.0894,  0.7485],
         [ 0.6137, -0.4419],
         [ 0.0984, -0.1520],
         [-0.5458,  0.9425],
         [ 0.3299,  1.5264],
         [-0.1864,  0.9193],
         [-0.3248,  1.3998]],

        [[-1.2415, -0.1413],
         [-0.5798, -0.3694],
         [-1.1544,  1.1677],
         [ 0.1884,  1.3586],
         [ 0.1845,  1.2366],
         [ 0.9801,  0.1697],
         [-0.9945,  1.1160],
         [ 0.9171,  1.7905]],

        [[ 0.7454, -0.2201],
         [-0.1093,  0.7529],
         [ 0.6874, -1.0833],
         [-0.8231, -0.7953],
         [ 0.9459, -0.9795],
         [ 0.1056, -1.1015],
         [-0.1636, -0.5730],
         [ 0.6923, -0.3152]],

        [[ 0.4694,  0.4213],
         [ 1.3748,  0.2873],
         [ 1.3502, -0.4173],
         [-1.2381,  0.5010],
         [-2.3167,  0.2943],
         [ 0.9573,  0.2935],
         [-1.6610, -1.7334],
         [ 1.9739, -0.9289]]])

In [ ]:
torch.manual_seed(2024)
B, T, C = 4, 8, 2
# x = torch.randn(B, T, C)

key = nn.Linear()

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei = wei / torch.sum(wei, 1, keepdim=True)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [ ]:
wei.shape

torch.Size([8, 8])

In [ ]:
xbow3.shape

torch.Size([4, 8, 2])

In [ ]:
xbow.shape

torch.Size([4, 8, 2])

In [ ]:
xbow[0]

tensor([[ 0.2914, -0.9938],
        [ 0.7004, -0.5584],
        [ 0.2362,  0.0455],
        [ 0.0811, -0.1823],
        [-0.1710, -0.3557],
        [-0.3103, -0.3775],
        [-0.2504, -0.4803],
        [ 0.0190, -0.3806]])

In [ ]:
xbow3[0]

tensor([[ 0.3171, -1.8716],
        [ 0.1138, -0.5615],
        [ 0.2804, -0.5216],
        [ 0.2349, -0.4292],
        [ 0.0788, -0.1549],
        [ 0.1206,  0.1253],
        [ 0.0768,  0.2388],
        [ 0.0266,  0.3839]])

In [ ]:
torch.manual_seed(2024)
B, T, C = 4, 8, 2
# x = torch.randn(B, T, C)

key = nn.Linear()

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei = wei / torch.sum(wei, 1, keepdim=True)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)